In [ ]:
%matplotlib inline

import time, os
import numpy as np
import matplotlib.pyplot as plt
import fitsio
import astropy.coordinates as coord
import astropy.units as u

In [ ]:
def getCounts(base, plan, wrap=True):
    files = os.listdir(base)
    res_files = [f for f in files if "{plan}-apo-fields".format(plan=plan) in f]
    counts = list()
    print("reading {} result files".format(len(res_files)))
    for f in res_files:
        sim_data = fitsio.read(base + f)
        counts.append(sim_data['nobservations'])
    
    fields = fitsio.read(base+"rsAllocation-{plan}-apo.fits".format(plan=plan))
    
    ra = coord.Angle(fields['racen']*u.degree)
    if wrap:
        ra = ra.wrap_at(180*u.degree)
    dec = coord.Angle(fields['deccen']*u.degree)
    
    planned = fields['needed']
    
    return ra, dec, counts, planned


def plotCompleteness(base, plan, extra=False):
    ra, dec, counts, planned = getCounts(base, plan)

    counts = np.mean(np.array(counts), axis=0)
    
    if extra:
        completeness = (counts - planned)/planned*100
    else:
        completeness = counts/planned*100
    
    idk = np.where(planned != 0)
    
    fig = plt.figure(figsize=(16,7))

    plt.subplot(projection="mollweide")

    if extra:
        image = plt.scatter(ra.radian[idk], dec.radian[idk], c=completeness[idk], 
                            cmap="viridis", vmax=50, vmin=-50, s=5)
        bar = fig.colorbar(image, orientation="vertical", pad=0.01)
        bar.set_label('% over planned', size=15)
        plt.savefig("field_completeness_extra.pdf")
    
    else:
        image = plt.scatter(ra.radian[idk], dec.radian[idk], c=completeness[idk], 
                            cmap="viridis", vmax=100, s=5)
        bar = fig.colorbar(image, orientation="vertical", pad=0.01)
        bar.set_label('% completed', size=15)
        plt.savefig("field_completeness.pdf")


def plotDeviation(base, plan, extra=False):
    ra, dec, counts, planned = getCounts(base, plan)

    if extra:
        counts = np.std(np.array(counts) - planned, axis=0)
    else:
        counts = np.std(np.array(counts), axis=0)

    idk = np.where(planned != 0)
#     idk = np.ones(len(counts), dtype=bool)

    fig = plt.figure(figsize=(16,7))

    plt.subplot(projection="mollweide")

    if extra:
        image = plt.scatter(ra.radian[idk], dec.radian[idk], c=counts[idk],
                            cmap="viridis", vmax=5, s=5)
        bar = fig.colorbar(image, orientation="vertical", pad=0.01)
        bar.set_label('deviation', size=15)

        plt.savefig("deviation_extra.pdf")

    else:
        image = plt.scatter(ra.radian[idk], dec.radian[idk], c=counts[idk],
                            cmap="viridis", vmax=5, s=5)
        bar = fig.colorbar(image, orientation="vertical", pad=0.01)
        bar.set_label('deviation', size=15)

        plt.savefig("deviation.pdf")

In [ ]:
base = os.path.expanduser("~/software/sdss/simCache/")
plan = "beta-1"
plotCompleteness(base, plan)
plotCompleteness(base, plan, extra=True)
plotDeviation(base, plan)

In [ ]:
# TO DO: analyze all output files. should be able to give us weather? 

lst_sum = np.genfromtxt("/home/john/software/sdss/simCache/lst_tracking.dat", names=True, delimiter=",")

# found that bug! no more "skipped" time
# missed = lst_sum["lst"][lst_sum["obs"] == -1]
# bins = np.arange(0,24, 1)
# plt.hist(missed/15, bins=bins)

In [ ]:
# this puts a bin edge at 24! so lsts between 23 and 24 can be included
bins = np.arange(0,25, 1)
plt.hist(lst_sum["lst"]/15, bins=bins, label="up")
plt.hist(lst_sum["obs"]/15, bins=bins, alpha=0.5, label="obs")
plt.legend(loc="best")

In [ ]:
fields = fitsio.read(base+"rsAllocation-{plan}-apo.fits".format(plan=plan))
    
ra = coord.Angle(fields['racen']*u.degree)

planned = fields['needed']
    
hist, bins, patches = plt.hist(lst_sum["lst"]/15, bins=bins, color="b", label="avail")
print("avail", np.sum(hist))
hist, bins, patches = plt.hist(lst_sum["obs"]/15, bins=bins, color="g", alpha=0.7, label="obs")
print("obs  ", np.sum(hist))
hist, bins, patches = plt.hist(ra/15, bins=bins, color="k", weights=planned, rwidth=0.5, label="plan")
print("plan ", np.sum(hist))
plt.legend(loc="best")
plt.savefig("lst_distribution.pdf")